In [1]:
import os
os.chdir('/home/jupyter/gitlab/advertorial-classifier/')

In [2]:
from advertorial import dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
#import wandb
import numpy as np
#import evaluate

/home/jupyter/venv/advertorial/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# # advertorial_dataset = dataset.train_valid_test_from_file(csv_file_path= './data/milelens_advertorial_dataset_formatted.csv')
# # train, validation, test = advertorial_dataset['train'], advertorial_dataset['validation'], advertorial_dataset['test'] 
# # id2label = {0: "no", 1: "yes"}
# # label2id = {"no": 0, "yes": 1}

# # pretrain_model ="hfl/chinese-bert-wwm-ext"
# # tokenizer = AutoTokenizer.from_pretrained(pretrain_model)
# # model = AutoModelForSequenceClassification.from_pretrained(
# #     pretrain_model, num_labels=2, id2label=id2label, label2id=label2id)



# advertorial_dataset = dataset.train_valid_test_from_file(csv_file_path='./data/milelens_advertorial_dataset_formatted.csv', train_ratio=0.8, validation_ratio=0.2 )#'./data/unseen_2023-05-30.csv', train_ratio=1)
# train = advertorial_dataset['train']
# valid = advertorial_dataset['validation']

# id2label = {0: "no", 1: "yes"}
# label2id = {"no": 0, "yes": 1}

# pretrain_model ="hfl/chinese-bert-wwm-ext"
# tokenizer = AutoTokenizer.from_pretrained(pretrain_model)
# model = AutoModelForSequenceClassification.from_pretrained(
#     pretrain_model, num_labels=2, id2label=id2label, label2id=label2id)

In [4]:
train_dataset = dataset.train_valid_test_from_file(csv_file_path='./data/train_set_2023-06-29.csv', train_ratio=1 )
valid_dataset = dataset.train_valid_test_from_file(csv_file_path='./data/valid_set_2023-06-29.csv', train_ratio=1 )
#'./data/unseen_2023-05-30.csv', train_ratio=1)
train = train_dataset['train']
valid = valid_dataset['train']
id2label = {0: "no", 1: "yes"}
label2id = {"no": 0, "yes": 1}

pretrain_model ="hfl/chinese-bert-wwm-ext"
tokenizer = AutoTokenizer.from_pretrained(pretrain_model)
model = AutoModelForSequenceClassification.from_pretrained(
    pretrain_model, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

In [5]:
from advertorial.inference import AdvertorialModel
adv = AdvertorialModel(model_path='./prebuilt_model/230629_chinese_bert_wwm_ext', use_gpu=True)

In [43]:
import pandas as pd
def perf_report(model, dataset, name='train'):
    from tqdm import tqdm
    N = len(dataset)
    step = 20
    ones = 0
    zeros = 0
    hits = 0
    miss = 0
    fp = 0
    fn = 0
    tp = 0
    tn = 0
    predictions = []
    for s in tqdm(range(0, N, step)):
        s, e = s, s+step
        prediction, probs = model(dataset[s:e]['text'])
        
        hits += np.sum(dataset[s:e]['label'] == prediction)
        miss += np.sum(dataset[s:e]['label'] != prediction)
        zeros += np.sum(dataset[s:e]['label'] == np.array(0))
        ones += np.sum(dataset[s:e]['label'] == np.array(1))
        fp += np.sum((dataset[s:e]['label'] != prediction) & (dataset[s:e]['label'] == np.array(1)))
        fn += np.sum((dataset[s:e]['label'] != prediction) & (dataset[s:e]['label'] == np.array(0)))
        tp += np.sum((dataset[s:e]['label'] == prediction) & (dataset[s:e]['label'] == np.array(1)))
        tn += np.sum((dataset[s:e]['label'] == prediction) & (dataset[s:e]['label'] == np.array(0)))
        predictions.append(prediction) 

    accuracy = hits/N
    print(f'accuracy:{accuracy:.2f}, positive samples:{ones}, negative samples:{zeros}')  
    performance_df = pd.DataFrame({'dataset':[name], 
                                   'records':[N], 
                                   'positive samples':[ones], 
                                   'negative samples':[zeros], 
                                   'hit':[hits],
                                   'miss':[miss],
                                   'accuracy':[accuracy], 
                                   'miss rate':[1-accuracy], 
                                   'false positive':[fp/(fp+fn)], 
                                   'false negative':[fn/(fp+fn)],
                                   'true positive':[tp/(tp+tn)],
                                   'true negative':[tn/(tp+tn)],
                                   'recall':[tp/(tp+fn)], 
                                   'precision':[tp/(tp+fp)]})

    predictions = np.concatenate(predictions)
    error_ids = predictions != dataset['label']
    error_df = pd.DataFrame({'text':np.array(dataset['text'])[error_ids], 'label':np.array(dataset['label'])[error_ids], 'prediction':predictions[error_ids]})
    return error_df, performance_df

In [44]:
train_error, train_perf = perf_report(adv, train, 'train')
validation_error, validation_perf = perf_report(adv, valid, 'validation')
#test_error, test_perf = perf_report(adv, test, 'test')

100%|██████████| 2176/2176 [12:49<00:00,  2.83it/s]


accuracy:0.99, positive samples:15229, negative samples:28275


100%|██████████| 544/544 [03:12<00:00,  2.83it/s]


accuracy:0.82, positive samples:3905, negative samples:6972


In [46]:
#pd.concat([train_perf, validation_perf, test_perf]).reset_index(drop=True)#.to_csv('performance.csv', index=False)
pd.concat([train_perf, validation_perf]).reset_index(drop=True).T#.to_csv('performance.csv', index=False)

,0,1
dataset,train,validation
records,43504,10877
positive samples,15229,3905
negative samples,28275,6972
hit,42980,8963
miss,524,1914
accuracy,0.987955,0.824032
miss rate,0.012045,0.175968
false positive,0.004528,0.09874
false negative,0.007517,0.077227


In [13]:
df = pd.read_excel('./data/業配文期末考題目.xlsx', sheet_name='Sheet1')
#df = df.rename(columns={'貼文內容':'text', '正確答案':'label'})[['text', 'label']]

In [17]:
join = train_error.merge(df, left_on='text', right_on='貼文內容', how='left')
join[['貼文內容', '正確答案', 'label', 'prediction', '貼文連結']].to_csv('業配文期末考題目_錯誤部分.csv', index=False)